In [1]:
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk 

In [2]:
#url_lists = ['https://www.dissentmagazine.org/online-articles/page/2']

In [3]:
url_lists=[]
for i in range(2,10):    
    url = "https://www.dissentmagazine.org/online-articles/page/{}".format(i)
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    #select only h3 tags with class flow-text and then select its child a 
    for j in soup.select("h3.flow-text > a"):
        article_url=j.get('href')
        url_lists.append(article_url)
        #print(j.get('href'))
soup.title

https://www.dissentmagazine.org/online-articles/page/2
https://www.dissentmagazine.org/online-articles/page/3
https://www.dissentmagazine.org/online-articles/page/4
https://www.dissentmagazine.org/online-articles/page/5
https://www.dissentmagazine.org/online-articles/page/6
https://www.dissentmagazine.org/online-articles/page/7
https://www.dissentmagazine.org/online-articles/page/8
https://www.dissentmagazine.org/online-articles/page/9


<title>Online Articles | Dissent Magazine</title>

In [4]:
len(url_lists)

96

In [5]:
def getpostlinks(url_lists):    
    for i in range(2,10): 
        url = "https://www.dissentmagazine.org/online-articles/page/{}".format(i)
        print(url)
        r = requests.get(url)
        soup = BeautifulSoup(r.content)
    
        #select only h3 tags with class flow-text and then select its child a 
        for j in soup.select("h3.flow-text > a"):
            article_url=j.get('href')
            url_lists.append(article_url)
    return

url_lists=[]
getpostlinks(url_lists)

https://www.dissentmagazine.org/online-articles/page/2
https://www.dissentmagazine.org/online-articles/page/3
https://www.dissentmagazine.org/online-articles/page/4
https://www.dissentmagazine.org/online-articles/page/5
https://www.dissentmagazine.org/online-articles/page/6
https://www.dissentmagazine.org/online-articles/page/7
https://www.dissentmagazine.org/online-articles/page/8
https://www.dissentmagazine.org/online-articles/page/9


In [6]:
def getpostText(Url):
    page = Request(Url, headers={'User-Agent': 'Mozilla/5.0'})
    html_page = urlopen(Url).read().decode('utf8','ignore') 
    soup = BeautifulSoup(html_page, 'html.parser')    
    for j in soup.findAll("section", {"id":'article'}):
        article=j.find_all('p') # select para with tag p inside section with id as article
    
        for k in article:
            text=k.get_text().replace("?"," ") # get the text from each para tag
            articlePosts.append(text)
            #print (text)
        
    return text

In [7]:
articlePosts = []
for link in url_lists:
    articlePosts+=getpostText(link)

In [8]:
articlePosts[:3]

['Benjamin Netanyahu has used the coronavirus to resuscitate his political career.',
 'On March 2, Israelis went to the polls for their country’s third national election in less than a year. Many of them hoped to vote out Prime Minister Benjamin Netanyahu, whose eleven consecutive\xa0years in power have been marked by a series of corruption scandals and an increasingly right-wing turn. At the time, the country was just waking up to the threat of coronavirus. Thousands of Israelis in quarantine received permission to leave their homes to vote in special polling tents. Everyone else cast their ballots as usual. Now, nearly a month later, most Israeli citizens are barred from traveling more than 100 meters (330 feet) from their homes, and Netanyahu has found a way to use the coronavirus situation to his advantage.',
 'Israel appears to be swiftly moving toward a ruling coalition led—at least initially—by Netanyahu, even though his right-wing bloc did not come out of the most recent electi

In [9]:
from stop_words import get_stop_words
#list(get_stop_words('en')

adhoc_list=['de', 'la', 'en', 'el', 'los', 'que', 'amlo', 'un', 'del', 'las','’','“','—','”','new','one','would','also','like','left','many','away','upon','de']
stop_word = set(stopwords.words('english') + list(get_stop_words('en'))+list(punctuation)+list(adhoc_list)+["'s","''","``","sign","newsletter"])
#stop_word

In [25]:
# instantiate a vectorizer object, within that specify stopwords
# min_df = 0.01 means "ignore terms that appear in less than 1% of the documents".
#max_df = 0.80 means "It ignores terms that appear in more than 80% of the documents".
vectorizer = TfidfVectorizer(max_df=.8,min_df=0.005,stop_words=stop_word)
# vectorizer fit_transform method takes a list of strings and then returns a two-dimensional array.Each row represents one document
X = vectorizer.fit_transform(articlePosts)
# So 6058 here represents the total number of articles & 686 distinct word in each articlein our corpus. 
X

<6058x686 sparse matrix of type '<class 'numpy.float64'>'
	with 44821 stored elements in Compressed Sparse Row format>

In [11]:
# The number of columns represents the number of distinct words which are present in all articles.
X[0]

<1x686 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [12]:
# X of zero will be a tuple of 686 numbers where each number represents the term frequency multiplied by inverse document frequency of one word.
print (X[0])

  (0, 640)	0.6157444642196925
  (0, 121)	0.6617006061155091
  (0, 449)	0.4277979226798229


In [13]:
# n_clusters specify the number of clusters,we want to divide our articles into groups. 
# The init specifies an algorithm to help choose the initial centroids in such a way that we can find the relevant clusters with a minimum number of iterations. 

k_means = KMeans(n_clusters = 4, init = 'k-means++', max_iter = 150, verbose = True)
k_means.fit(X)


Initialization complete
Iteration  0, inertia 2505.713
Iteration  1, inertia 2465.477
Iteration  2, inertia 2457.493
Iteration  3, inertia 2456.265
Iteration  4, inertia 2455.997
Iteration  5, inertia 2455.943
Converged at iteration 5: center shift 0.000000e+00 within tolerance 5.974508e-08
Initialization complete
Iteration  0, inertia 2508.779
Iteration  1, inertia 2471.064
Iteration  2, inertia 2465.691
Iteration  3, inertia 2463.513
Iteration  4, inertia 2462.878
Iteration  5, inertia 2462.386
Iteration  6, inertia 2462.261
Iteration  7, inertia 2462.246
Iteration  8, inertia 2462.233
Converged at iteration 8: center shift 0.000000e+00 within tolerance 5.974508e-08
Initialization complete
Iteration  0, inertia 2413.169
Iteration  1, inertia 2369.728
Iteration  2, inertia 2356.471
Iteration  3, inertia 2353.837
Iteration  4, inertia 2353.106
Iteration  5, inertia 2352.889
Iteration  6, inertia 2352.721
Iteration  7, inertia 2352.708
Iteration  8, inertia 2352.692
Iteration  9, inerti

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=150,
    n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [14]:
# Every document in array X has now been assigned a number, which represents the cluster to which it belongs
# So cluster zero has 76 articles. Cluster 1 has 86 and cluster 2 has 5794 articles
np.unique(k_means.labels_, return_counts=True)

(array([0, 1, 2, 3]), array([  76,   86, 5794,  102]))

In [15]:
# set up a dictionary 'text' in which the keys will be the cluster numbers and the values will be the aggregated text across all the articles which are present
text={}
for i,cluster in enumerate(k_means.labels_):
    article_tag = articlePosts[i]
    if cluster not in text.keys():
        text[cluster] = article_tag
    else:
        text[cluster] += article_tag

In [16]:
# iterate through each cluster and take the corresponding text and tokenize it into words.filter out all the stop words
# pick the top 100 words using frequency distribution and get corrsponding counts
keywords = {}
word_counts={}
for cluster in range(3):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent=[word for word in word_sent if word not in stop_word]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100, freq, key=freq.get)
    word_counts[cluster]=freq

In [17]:
# find the 10 keywords which are unique to each cluster
unique_keys={}
for cluster in range(3):   
    other_clusters=list(set(range(3))-set([cluster]))
    keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique=set(keywords[cluster])-keys_other_clusters
    unique_keys[cluster]=nlargest(10, unique, key=word_counts[cluster].get)

In [18]:
unique_keys

{0: ['housing',
  'rent',
  'land',
  'tenants',
  'development',
  'homes',
  'supply',
  'market',
  'prices',
  'affordability'],
 1: ['hong',
  'kong',
  'china',
  'chinese',
  'international',
  'protests',
  'beijing',
  'bill',
  'police',
  'protesters'],
 2: ['power',
  'trump',
  'states',
  'party',
  'united',
  'workers',
  'politics',
  'national',
  'history',
  'american']}

In [19]:
snippet = "A group of 30 Kenyan runners from across the country ran solo half marathons to compete in a virtual race amid coronavirus pandemic. They had been training for Wuhan Marathon which was to be held this week but got postponed. As a part of CoronaRun organised in Kenya, runners ran a 21-km course and recorded their time using GPS watches."

In [20]:
# fit method to set up the training phase to parse in the complete set of articles for which the labels are already known.
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X,k_means.labels_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [21]:
# used vectorizer. transform to convert the article into the TF-IDF form.
sample=vectorizer.transform([snippet])

In [22]:
sample

<1x686 sparse matrix of type '<class 'numpy.float64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [23]:
# we can use the predict method of the classifier to assign a theme for snippet
classifier.predict(sample)

array([2])